In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geoband.API import *
import statsmodels.api as sm
import folium
from folium import plugins
import json
from sklearn.preprocessing import StandardScaler

In [2]:
final = pd.read_csv('finalll.txt', delimiter = '\t', encoding = 'euc-kr')

In [3]:
GetCompasData('SBJ_2012_001', '2', '2.오산시_어린이교통사고_격자.geojson')
data2 = gpd.read_file('2.오산시_어린이교통사고_격자.geojson')

In [4]:
del data2['accident_cnt']

In [5]:
del final['Unnamed: 0']
del final['geometry']

In [6]:
final = pd.merge(final, data2, on='gid')

In [7]:
final = gpd.GeoDataFrame(final, geometry = final.geometry)

In [8]:
center = final.geometry.centroid
center = pd.DataFrame(data = center, columns = ['center'])

In [9]:
final = pd.concat((final, center), axis=1)

In [10]:
final.set_crs(epsg=4326, inplace=True)

,gid,accident_cnt,schoolzone_num,safetysign_num,bump_num,cctv_num,child_num,school_num,facility_num,signal_num,...,car_num,node,park_yn,foot_traf_mean,traffic,bigcar_ratio,rushhour_freq,road_width,geometry,center
0,다사563097,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,33.136750,4723.430,0.232105,3.9750,0.500,"MULTIPOLYGON (((127.00766 37.18506, 127.00766 ...",POINT (127.00822 37.18551)
1,다사563098,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,0.000000,4723.430,0.232105,3.9750,0.500,"MULTIPOLYGON (((127.00766 37.18596, 127.00765 ...",POINT (127.00822 37.18641)
2,다사564095,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,12.132188,4835.990,0.232096,5.2850,0.500,"MULTIPOLYGON (((127.00880 37.18326, 127.00880 ...",POINT (127.00936 37.18371)
3,다사564096,1,0,1,0,1,0.0,0.0,0.0,0,...,0,1,0,42.714375,4779.710,0.232100,4.6300,0.500,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ...",POINT (127.00936 37.18461)
4,다사564097,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,25.259604,4723.430,0.232105,3.9750,0.500,"MULTIPOLYGON (((127.00879 37.18506, 127.00878 ...",POINT (127.00935 37.18551)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,다사640065,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,2.442500,1936.405,0.117912,29.0625,0.625,"MULTIPOLYGON (((127.09457 37.15654, 127.09456 ...",POINT (127.09513 37.15699)
1710,다사640066,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,171.749722,2293.930,0.057351,19.6000,0.500,"MULTIPOLYGON (((127.09456 37.15744, 127.09456 ...",POINT (127.09512 37.15789)
1711,다사640067,0,0,0,0,0,0.0,0.0,0.0,0,...,0,0,0,0.000000,2293.930,0.057351,19.6000,0.500,"MULTIPOLYGON (((127.09456 37.15834, 127.09455 ...",POINT (127.09512 37.15880)
1712,다사641063,0,0,0,0,0,0.0,0.0,0.0,0,...,0,1,0,9.059167,1578.880,0.178473,38.5250,0.750,"MULTIPOLYGON (((127.09570 37.15474, 127.09570 ...",POINT (127.09626 37.15519)


In [11]:
final['lat'] = final['center'].apply(lambda coord: coord.y)
final['lon'] = final['center'].apply(lambda coord: coord.x)
del final['center']

In [12]:
del final['geometry']

In [13]:
x_list = [2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

In [14]:
X_train = final.iloc[:, x_list]

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

In [16]:
coords = np.array(list(zip(final['lat'], final['lon'])))

In [17]:
y = np.array(final.accident_cnt).reshape((-1,1))

In [18]:
training_results = sm.ZeroInflatedPoisson(endog = y, exog = X_train, inflation = 'logit').fit()

/opt/app-root/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 0.644406
         Iterations: 21
         Function evaluations: 23
         Gradient evaluations: 23


In [19]:
print(training_results.summary())

                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:                       y   No. Observations:                 1714
Model:             ZeroInflatedPoisson   Df Residuals:                     1695
Method:                            MLE   Df Model:                           18
Date:                 Mon, 25 Jan 2021   Pseudo R-squ.:                 0.07899
Time:                         14:21:39   Log-Likelihood:                -1104.5
converged:                        True   LL-Null:                       -1199.2
Covariance Type:             nonrobust   LLR p-value:                 1.279e-30
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const     0.7689      0.073     10.584      0.000       0.626       0.911
x1               -0.0059      0.046     -0.127      0.899      -0.097       0.085
x2                0.0774      0.

In [20]:
predictions = training_results.predict(X_train)

In [21]:
predicted_counts = np.round(predictions)

In [22]:
print('RMSE='+str(np.sqrt(np.sum(np.power(np.subtract(predicted_counts,y),2)))))

RMSE=1491.437561549259


In [23]:
predicted_counts = pd.DataFrame(data = predicted_counts, columns = ['predicted_counts'])

In [24]:
temp = pd.read_csv('finalll.txt', delimiter = '\t', encoding = 'euc-kr')
temp = temp.iloc[:,[1,3]]
zip_df = pd.concat((temp, pd.DataFrame(predicted_counts)), axis=1)
zip_df = pd.merge(zip_df, data2, on='gid')
zip_df = gpd.GeoDataFrame(zip_df, geometry = zip_df.geometry)
zip_df = zip_df.set_crs(epsg=4326)

In [25]:
converted_json = zip_df.to_json()



m = folium.Map(
    location = [37.16323633764299, 127.05128286756862],
    tiles = 'http://api.vworld.kr/req/wmts/1.0.0/A0D28BDF-548E-38C9-BFE4-ACBC9CF80602/Satellite/{z}/{y}/{x}.jpeg', 
    attr = '오산시',
    zoom_start=14
)



choropleth = folium.Choropleth(
    geo_data=converted_json,
    data = pd.DataFrame(zip_df),
    columns = ('gid','predicted_counts'),
    key_on = 'feature.properties.gid',
    fill_color = 'YlOrRd',
    legend_name = '사고수',
    nan_fill_color = 'blue'
).add_to(m)


#Popup창 뜨게 하기
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['gid','accident_cnt', 'predicted_counts'], labels = True)
)

m